In [245]:
%load_ext autoreload
%autoreload 2

from src.base_classes.omic_data_loader import OmicDataLoader
from src.data_managers.concat import CatOmicDataManager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [246]:
mrna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74_disease/mrna",
)
mirna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74_disease/mirna_genes",
)
circrna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74_disease/circrna",
)
# pirna_loader = OmicDataLoader(
#     data_dir="mds_data/splits_74/pirna",
# )
te_loader = OmicDataLoader(
    data_dir="mds_data/splits_74_disease/te_counts",
)

In [247]:
omic_data_loaders = {
    "mrna": mrna_loader,
    "mirna": mirna_loader,
    "circrna": circrna_loader,
    # # "pirna": pirna_loader,
    "te": te_loader,
}
odm = CatOmicDataManager(omic_data_loaders, n_splits=5)

save_folder = f"logs/mds_disease/{'_'.join(omic_data_loaders.keys())}.csv"
save_folder

'logs/mds_disease/mrna_mirna_circrna_te.csv'

In [248]:
# from src.evals.knn import KNNEvaluator

# knn_eval = KNNEvaluator(
#     data_manager=odm,
#     n_trials=20,
#     verbose=True,
#     params={"k_lb": 1, "k_ub": 20},
# )
# _ = knn_eval.evaluate()
# knn_eval.save_results(results_file=save_folder, row_name="KNN")

In [249]:
# from src.evals.svm import SVMEvaluator

# svm_eval = SVMEvaluator(
#     data_manager=odm,
#     n_trials=50,
#     verbose=True,
#     params={
#         "C_lb": 0.01,
#         "C_ub": 10,
#         "no_rfe": True,
#         # "rfe_step_range": (0.05, 0.2),
#         # "rfe_n_features_range": (100, 200),
#     },
#     mode="linear",
# )
# _ = svm_eval.evaluate()
# svm_eval.save_results(results_file=save_folder, row_name="Linear SVM")

In [250]:
from src.evals.xgboost import XGBoostEvaluator

xgb_eval = XGBoostEvaluator(
    data_manager=odm,
    n_trials=60,
    verbose=True,
)
_ = xgb_eval.evaluate()
xgb_eval.print_best_results()
xgb_eval.save_results(results_file=save_folder, row_name="XGBoost")

[I 2024-11-18 21:47:41,432] A new study created in memory with name: no-name-93ef448d-4658-4818-98c9-91212fcce708
[I 2024-11-18 21:47:41,689] Trial 0 finished with value: 0.6862235118921005 and parameters: {'lambda': 2.1038579239515384e-06, 'alpha': 0.05141341421311622}. Best is trial 0 with value: 0.6862235118921005.


New best score: 0.686
Best model performance:
Accuracy: 0.920 ± 0.050
F1 Macro: 0.824 ± 0.109
F1 Weighted: 0.906 ± 0.061


[I 2024-11-18 21:47:41,901] Trial 1 finished with value: 0.6631836987350632 and parameters: {'lambda': 1.2088820012011197e-07, 'alpha': 0.0011091066773955124}. Best is trial 0 with value: 0.6862235118921005.
[I 2024-11-18 21:47:42,114] Trial 2 finished with value: 0.6631836987350632 and parameters: {'lambda': 1.659749578254492e-07, 'alpha': 0.00359848388031159}. Best is trial 0 with value: 0.6862235118921005.
[I 2024-11-18 21:47:42,339] Trial 3 finished with value: 0.6631836987350632 and parameters: {'lambda': 7.835330852592804e-07, 'alpha': 0.004539914885993437}. Best is trial 0 with value: 0.6862235118921005.
[I 2024-11-18 21:47:42,548] Trial 4 finished with value: 0.6460639796087643 and parameters: {'lambda': 2.267567440577727e-08, 'alpha': 0.023732338304611578}. Best is trial 0 with value: 0.6862235118921005.
[I 2024-11-18 21:47:42,754] Trial 5 finished with value: 0.5555740074333481 and parameters: {'lambda': 6.759802106598952e-07, 'alpha': 0.07162835145059197}. Best is trial 0 wi

New best score: 0.775
Best model performance:
Accuracy: 0.932 ± 0.042
F1 Macro: 0.891 ± 0.067
F1 Weighted: 0.933 ± 0.042


[I 2024-11-18 21:47:43,622] Trial 8 finished with value: 0.27794757275052406 and parameters: {'lambda': 2.446861228203048e-08, 'alpha': 0.26662492542896093}. Best is trial 7 with value: 0.775476069168414.
[I 2024-11-18 21:47:43,880] Trial 9 finished with value: 0.27794757275052406 and parameters: {'lambda': 0.0002117912938432218, 'alpha': 0.4403368801388419}. Best is trial 7 with value: 0.775476069168414.
[I 2024-11-18 21:47:44,103] Trial 10 finished with value: 0.7039337563024542 and parameters: {'lambda': 0.26335951529044704, 'alpha': 1.4199788267061524e-08}. Best is trial 7 with value: 0.775476069168414.
[I 2024-11-18 21:47:44,347] Trial 11 finished with value: 0.7581635155117249 and parameters: {'lambda': 0.08503108386645856, 'alpha': 1.9477082034432167e-08}. Best is trial 7 with value: 0.775476069168414.
[I 2024-11-18 21:47:44,573] Trial 12 finished with value: 0.7581635155117249 and parameters: {'lambda': 0.00261108583761937, 'alpha': 1.171997708522742e-08}. Best is trial 7 with 

New best score: 0.814
Best model performance:
Accuracy: 0.946 ± 0.050
F1 Macro: 0.910 ± 0.080
F1 Weighted: 0.946 ± 0.050


[I 2024-11-18 21:47:46,371] Trial 20 finished with value: 0.6631836987350632 and parameters: {'lambda': 0.0005110446110602179, 'alpha': 0.00028752061884983004}. Best is trial 19 with value: 0.8139129397754031.
[I 2024-11-18 21:47:46,598] Trial 21 finished with value: 0.7415719661868713 and parameters: {'lambda': 1.7547542456180452e-05, 'alpha': 7.493126388860602e-08}. Best is trial 19 with value: 0.8139129397754031.
[I 2024-11-18 21:47:46,823] Trial 22 finished with value: 0.8139820349629631 and parameters: {'lambda': 0.0007898174161379283, 'alpha': 1.0471281466017934e-07}. Best is trial 22 with value: 0.8139820349629631.


New best score: 0.814
Best model performance:
Accuracy: 0.947 ± 0.050
F1 Macro: 0.910 ± 0.080
F1 Weighted: 0.945 ± 0.051


[I 2024-11-18 21:47:47,086] Trial 23 finished with value: 0.7581635155117249 and parameters: {'lambda': 0.0008172770090024554, 'alpha': 3.6398615664642408e-06}. Best is trial 22 with value: 0.8139820349629631.
[I 2024-11-18 21:47:47,308] Trial 24 finished with value: 0.7415719661868713 and parameters: {'lambda': 4.052705675563501e-06, 'alpha': 1.419519966419499e-07}. Best is trial 22 with value: 0.8139820349629631.
[I 2024-11-18 21:47:47,534] Trial 25 finished with value: 0.7334652041272984 and parameters: {'lambda': 0.00021879900700020848, 'alpha': 2.4702272653377105e-06}. Best is trial 22 with value: 0.8139820349629631.
[I 2024-11-18 21:47:47,758] Trial 26 finished with value: 0.7581635155117249 and parameters: {'lambda': 0.002640741228750489, 'alpha': 7.218483584767016e-08}. Best is trial 22 with value: 0.8139820349629631.
[I 2024-11-18 21:47:47,987] Trial 27 finished with value: 0.6305302169311957 and parameters: {'lambda': 9.299144082936562e-05, 'alpha': 2.1025832043402485e-05}. B

Best model performance:
Accuracy: 0.947 ± 0.050
F1 Macro: 0.910 ± 0.080
F1 Weighted: 0.945 ± 0.051
Results saved to logs/mds_disease/mrna_mirna_circrna_te.csv


In [251]:
# from src.evals.mlp import MLPEvaluator

# mlp_eval = MLPEvaluator(
#     data_manager=odm,
#     n_trials=10,
#     verbose=True,
#     params={
#         "lr_range": [1e-4, 1e-2],
#         "l2_lambda": 5e-4,
#         "dropout_range": [0.1, 0.6],
#         "hidden_channels": 64,
#         "proj_dim": 64,
#         "batch_size": 32,
#         "max_epochs": 50,
#     },
# )
# _ = mlp_eval.evaluate()
# mlp_eval.save_results(results_file=save_folder, row_name="MLP")

In [252]:
# mlp_eval.print_best_results()
# mlp_eval.print_best_parameters()

In [253]:
from src.evals.mogonet import MOGONETEvaluator
from src.data_managers.sample_graph import SampleGraphDataManager

mogonet_eval = MOGONETEvaluator(
    data_manager=SampleGraphDataManager(
        omic_data_loaders=omic_data_loaders,
        n_splits=5,
        params={
            "graph_style": "threshold",
            "self_connections": True,
            "avg_degree": 5,
        },
    ),
    n_trials=1,
    params={
        "encoder_hidden_channels": {
            "mrna": 64,
            "mirna": 64,
            "circrna": 64,
            "te": 64,
        },
        "encoder_type": "gat",
        "dropout": 0.2,
        "integrator_type": "vcdn", # vcdn seems to work better on mds disease
        "integration_in_dim": 16,
        "vcdn_hidden_channels": 16,
        "epochs": 250,
        "log_interval": 251,
    }
)
mogonet_eval.evaluate()
mogonet_eval.print_best_results()
mogonet_eval.save_results(results_file=save_folder, row_name="MOGONET")

[I 2024-11-18 21:47:55,371] A new study created in memory with name: no-name-27f7832d-843c-4b18-a274-fedc36a796b7


Using: vcdn integrator
Using: vcdn integrator
Using: vcdn integrator
Using: vcdn integrator
Using: vcdn integrator


[I 2024-11-18 21:52:40,261] Trial 0 finished with value: 0.7767231525925926 and parameters: {}. Best is trial 0 with value: 0.7767231525925926.


New best score: 0.777
Best model performance:
Accuracy: 0.933 ± 0.060
F1 Macro: 0.893 ± 0.093
F1 Weighted: 0.932 ± 0.060
Best model performance:
Accuracy: 0.933 ± 0.060
F1 Macro: 0.893 ± 0.093
F1 Weighted: 0.932 ± 0.060


# linear
Accuracy: 0.947 ± 0.050
F1 Macro: 0.910 ± 0.080
F1 Weighted: 0.945 ± 0.051
# vcdn
- integration dim = 2
Accuracy: 0.825 ± 0.030
F1 Macro: 0.452 ± 0.009
F1 Weighted: 0.746 ± 0.043
- integration dim = 8
Accuracy: 0.973 ± 0.033
F1 Macro: 0.952 ± 0.059
F1 Weighted: 0.971 ± 0.035
- integration dim = 12
Accuracy: 0.973 ± 0.033
F1 Macro: 0.957 ± 0.053
F1 Weighted: 0.973 ± 0.033
- integration dim = 16
Accuracy: 0.973 ± 0.053
F1 Macro: 0.958 ± 0.083
F1 Weighted: 0.973 ± 0.053
# attention - faster than vcdn
- integration dim = 2
Accuracy: 0.933 ± 0.060
F1 Macro: 0.877 ± 0.114
F1 Weighted: 0.927 ± 0.067
- integration dim = 8
Accuracy: 0.973 ± 0.033
F1 Macro: 0.952 ± 0.059
F1 Weighted: 0.971 ± 0.035
- integration dim = 12
Accuracy: 0.947 ± 0.050
F1 Macro: 0.910 ± 0.080
F1 Weighted: 0.945 ± 0.051
- integration dim = 16
Accuracy: 0.960 ± 0.053
F1 Macro: 0.934 ± 0.085
F1 Weighted: 0.959 ± 0.054

In [254]:
from src.evals.birgat import BiRGATEvaluator
from src.data_managers.bipartite_graph import BipartiteGraphDataManager

three_layers = True

birgat_eval = BiRGATEvaluator(
    data_manager=BipartiteGraphDataManager(
        omic_data_loaders=omic_data_loaders,
        n_splits=5,
        params={
            "diff_exp_thresholds" : {
                "mrna": 1.5,
                "mirna": 1.5,
                "circrna": 1.7,
                "te": 1.7,
            },
        },
    ),
    params={
        "epochs": 350,
        "log_interval": 50,
        "hidden_channels": [200, 64, 64, 16, 16],
        "heads": 4,
        "dropout": 0.2,
        "attention_dropout": 0.0,
        "integrator_type": "vcdn",
        "three_layers": three_layers,
    },
    n_trials=1,
)
birgat_eval.evaluate()
birgat_eval.print_best_results()
birgat_eval.save_results(results_file=save_folder, row_name="BiRGAT 3L" if three_layers else "BiRGAT 2L")

isolated sample nodes, isolated gene nodes, mean degree: 
tensor(1) tensor(0) tensor(25.6892)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(0) tensor(0) tensor(21.1486)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(0) tensor(9) tensor(16.1351)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(4) tensor(0) tensor(15.5676)


[I 2024-11-18 21:52:40,696] A new study created in memory with name: no-name-9103e056-f51b-405b-a835-ada329d5a443


isolated sample nodes, isolated gene nodes, mean degree: 
tensor(1) tensor(0) tensor(25.6892)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(0) tensor(0) tensor(21.1486)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(0) tensor(9) tensor(16.1351)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(4) tensor(0) tensor(15.5676)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(1) tensor(0) tensor(25.6892)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(0) tensor(0) tensor(21.1486)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(0) tensor(9) tensor(16.1351)
isolated sample nodes, isolated gene nodes, mean degree: 
tensor(4) tensor(0) tensor(15.5676)

Epoch: 050:
Train Loss: 0.4944, Train Acc: 0.8305, Train F1 Macro: 0.4537, Train F1 Weighted: 0.7536
Val Acc: 0.8000, Val F1 Macro: 0.4444, Val F1 Weighted: 0.7111, Val Geometric Mean: 0.6323
Test Acc: 0.8000, Test F1 Macro: 0.4444, Test F1 Weight

[I 2024-11-18 21:59:24,235] Trial 0 finished with value: 0.6932284788414499 and parameters: {}. Best is trial 0 with value: 0.6932284788414499.



Epoch: 350:
Train Loss: 0.0431, Train Acc: 0.9833, Train F1 Macro: 0.9731, Train F1 Weighted: 0.9836
Val Acc: 0.7857, Val F1 Macro: 0.4400, Val F1 Weighted: 0.7543, Val Geometric Mean: 0.6389
Test Acc: 0.7857, Test F1 Macro: 0.4400, Test F1 Weighted: 0.7543
##################################################
New best score: 0.693
Best model performance:
Accuracy: 0.931 ± 0.045
F1 Macro: 0.813 ± 0.185
F1 Weighted: 0.916 ± 0.068
Best model performance:
Accuracy: 0.931 ± 0.045
F1 Macro: 0.813 ± 0.185
F1 Weighted: 0.916 ± 0.068


In [255]:
# birgat_eval.save_results(results_file=save_folder, row_name="BiRGAT")
save_folder

'logs/mds_disease/mrna_mirna_circrna_te.csv'

```python
omic_data_loaders = {
    "mrna": mrna_loader,
    "mirna": mirna_loader,
    "circrna": circrna_loader,
    # # "pirna": pirna_loader,
    # "te": te_loader,
}

birgat_eval = BiRGATEvaluator(
    data_manager=BipartiteGraphDataManager(
        omic_data_loaders=omic_data_loaders,
        n_splits=5,
        params={
            "diff_exp_thresholds" : {
                "mrna": 1.5,
                "mirna": 1.5,
                "circrna": 1.7,
                # "te": 1.8,
            },
        },
    ),
    params={
        "epochs": 250,
        "log_interval": 50,
        "hidden_channels": [200, 64, 64, 16, 16],
        "heads": 4,
        "dropout": 0.2,
        "attention_dropout": 0.0,
        "integrator_type": "vcdn",
        "three_layers": False,
    },
    n_trials=1,
)
```

Accuracy: 0.960 ± 0.053
F1 Macro: 0.940 ± 0.082
F1 Weighted: 0.961 ± 0.053

# mrna, mirna, circrna, 2L
Accuracy: 0.960 ± 0.033
F1 Macro: 0.915 ± 0.073
F1 Weighted: 0.956 ± 0.036
# mrna, mirna, circrna, 2L no interactions
Accuracy: 0.960 ± 0.053
F1 Macro: 0.918 ± 0.113
F1 Weighted: 0.953 ± 0.064
---
Accuracy: 0.946 ± 0.027
F1 Macro: 0.904 ± 0.048
F1 Weighted: 0.944 ± 0.028
# mrna, mirna, circrna 3L, interactions, degree ~20 in diff exp graphs, larger degree shows degraded performance
# making the avg degree to high shows large jumps on the validation set during training
Accuracy: 0.945 ± 0.053
F1 Macro: 0.910 ± 0.081
F1 Weighted: 0.946 ± 0.048
# mrna, mirna, circrna 2L, interactions, 64 cap
Accuracy: 0.960 ± 0.053
F1 Macro: 0.940 ± 0.082
F1 Weighted: 0.961 ± 0.053
# mrna, mirna, circrna 3L, interactions, 64 cap
Accuracy: 0.891 ± 0.054
F1 Macro: 0.801 ± 0.088
F1 Weighted: 0.888 ± 0.056
# mrna, mirna, circrna 3L
Accuracy: 0.920 ± 0.050
F1 Macro: 0.829 ± 0.112
F1 Weighted: 0.907 ± 0.062
# mrna, mirna, 2L
Accuracy: 0.960 ± 0.033
F1 Macro: 0.915 ± 0.073
F1 Weighted: 0.956 ± 0.036
# mrna, mirna, 3L
Accuracy: 0.947 ± 0.050
F1 Macro: 0.897 ± 0.089
F1 Weighted: 0.944 ± 0.051